In [11]:
import pandas as pd
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
import numpy as np
from sklearn.model_selection import StratifiedKFold
import os

In [15]:
# 更改
dire = "E:\工作\模型\人行模型"
os.chdir(dire)

In [4]:
def peek(df, line=None):
    print(df.shape)
    if line is None:
        print(df.head())
    else:
        print(df.head(line))

根据初步探索，做如下处理
- dm_id_003用NONE取代空值
- dm_id_005用NONE取代空值
- dm_id_009用NONE取代空值
- dm_creditdetails_007转化成比例，以及拆开两列
- 这些字段需要做这个变换：取整((原始数值-987) * 100)*10，dm_summary_002dm_summary_005，dm_summary_006，dm_summary_007，dm_summary_011，dm_summary_012，dm_summary_014，dm_summary_018，dm_creditdetails_031，dm_creditdetails_032，dm_creditdetails_034，dm_publicdetails_009，dm_creditdetails_028
- 做对数转换的变量：dm_summary_002, dm_summary_005, dm_summary_006, dm_summary_007, dm_summary_011, dm_summary_012 (ds_011和012两个变量不是整数，而且最大值较少，问一下原因), dm_summary_014, dm_summary_018, dm_credidetails_001, dm_creditdetails_003, dm_creditdetails_004, dm_creditdetails_005, dm_creditdetails_006, dm_creditdetails_030, dm_creditdetails_031, dm_creditdetails_032(每月为什么最大值这么少）, dm_creditdetails_034
- dm_summary_021 转换成boolean, 非0数据太少，转化意义不大，暂不转换
- dm_summary_022, dm_publicdetails_001, dm_publicdetails_002, dm_publicdetails_003, dm_creditdetails_024 全部值为0，丢弃
- dm_summary_023转换成有代偿和无代偿，非0数据太少，转化意义不大，暂不转换
- dm_summary_024转换为有查询和无查询，非0数据太少，转化意义不大，暂不转换
- dm_publicdetails_008剔除异常值1474
- dm_repayment*字段转化为逾期占比

In [16]:
hf_parsed = pd.read_csv("./data/hf_success_parsed.csv")

In [17]:
loan_good = pd.read_csv("./data/loan_good_202302241703.csv")
loan_good['label'] = 0
loan_bad = pd.read_csv("./data/loan_bad_202302241702.csv")
loan_bad['label'] = 1
loan = pd.concat([loan_good, loan_bad])
loan_user = pd.read_csv("./data/loan_user_202302241703.csv")
loan = loan.merge(loan_user, on = ['user_id'], how='left')
loan = loan.query('product_name != "灰度测试"')
loan = loan[['customer_unique_key', 'label']].drop_duplicates()
peek(loan, 2)

(813, 2)
           customer_unique_key  label
7  C20220720205066554575032370      0
8  C20220721205244862285615186      0


In [9]:
hf_parsed = hf_parsed.merge(loan[['customer_unique_key', 'label']], on = 'customer_unique_key', how='inner')
peek(hf_parsed, 2)

(567, 85)
                    id          customer_unique_key  \
0  1546580974007521280  C20230102265075015786049564   
1  1542599547565936640  C20221222261089915742339083   

                     serial_no              create_time            edit_time  \
0  S20230102265075392744927233  2023-01-02T11:14:58.791  2023-01-03T00:30:23   
1  S20221222261093966475309101  2022-12-22T11:34:12.756  2022-12-23T00:31:08   

  dm_id_003 dm_id_005 dm_id_009  dm_summary_001  dm_summary_002  \
0        GZ        YH        ZZ               1         987.578   
1       NaN       NaN       NaN               0           0.000   

   dm_summary_003  dm_summary_004  dm_summary_005  dm_summary_006  \
0               1               5       1003.3572       1017.2184   
1               0               0          0.0000          0.0000   

   dm_summary_007  dm_summary_011  dm_summary_012  dm_summary_013  \
0       1029.4385       1010.4735             0.0               0   
1          0.0000          0.0000  

In [10]:
hf_parsed['dm_id_003'] = hf_parsed['dm_id_003'].fillna('NONE')
print(hf_parsed['dm_id_003'].value_counts())
hf_parsed['dm_id_005'] = hf_parsed['dm_id_005'].fillna('NONE')
print(hf_parsed['dm_id_005'].value_counts())
hf_parsed['dm_id_009'] = hf_parsed['dm_id_009'].fillna('NONE')
print(hf_parsed['dm_id_009'].value_counts())

NONE    284
GZ       76
DZ       70
ZZ       46
XX       43
BK       26
QT       20
YJS       2
Name: dm_id_003, dtype: int64
NONE    366
YH      201
Name: dm_id_005, dtype: int64
NONE    281
ZZ      192
ZY       35
QT       25
GTJY     13
ZYJS     10
GR        4
GWY       3
NM        3
QYGL      1
Name: dm_id_009, dtype: int64


In [42]:
hf_parsed[['dm_creditdetails_007_numerator', 'dm_creditdetails_007_denominator']] = \
                            hf_parsed['dm_creditdetails_007'].replace("-9990996", np.nan).str.split("/", expand=True)
hf_parsed['dm_creditdetails_007_numerator'] = hf_parsed['dm_creditdetails_007_numerator'].astype("float")
hf_parsed['dm_creditdetails_007_denominator'] = hf_parsed['dm_creditdetails_007_denominator'].astype("float")
hf_parsed['dm_creditdetails_007'] = hf_parsed['dm_creditdetails_007_numerator'] / hf_parsed['dm_creditdetails_007_denominator']
hf_parsed['dm_creditdetails_007'] = hf_parsed['dm_creditdetails_007_denominator'].fillna(0)
hf_parsed = hf_parsed.drop(columns=['dm_creditdetails_007_denominator'])
# peek(hf_parsed, 2)

In [61]:
need_reverse_vars = ['dm_summary_002', 'dm_summary_005', 'dm_summary_006', 'dm_summary_007', 'dm_summary_011','dm_summary_012','dm_summary_014','dm_summary_018',
                     'dm_creditdetails_031','dm_creditdetails_032','dm_creditdetails_034','dm_publicdetails_009','dm_creditdetails_028']

hf_parsed[need_reverse_vars] = hf_parsed[need_reverse_vars].applymap(lambda x: int((x- 987) * 100) * 10 if x > 0 else 0)

In [65]:
need_log_vars = ['dm_summary_002', 'dm_summary_005', 'dm_summary_006', 'dm_summary_007', 'dm_summary_011', 'dm_summary_012', 'dm_summary_014', 'dm_summary_018', 'dm_creditdetails_001', 
                 'dm_creditdetails_003', 'dm_creditdetails_004', 'dm_creditdetails_005', 'dm_creditdetails_006', 'dm_creditdetails_030', 'dm_creditdetails_031',
                 'dm_creditdetails_032', 'dm_creditdetails_034']

In [69]:
hf_parsed[need_log_vars] = hf_parsed[need_log_vars].applymap(np.log1p)
hf_parsed.shape

(567, 86)

In [85]:
hf_parsed['dm_repayment_1_001'] = hf_parsed['dm_repayment_1_001'].fillna("/" * 24)

In [86]:
hf_parsed['dm_repayment_15_001'] = hf_parsed['dm_repayment_15_001'].fillna("/" * 60)

In [87]:
hf_parsed['dm_repayment_2_001'] = hf_parsed['dm_repayment_2_001'].fillna("/" * 24)

In [88]:
hf_parsed['dm_repayment_25_001'] = hf_parsed['dm_repayment_25_001'].fillna("/" * 60)

In [117]:
for c in hf_parsed.filter(regex="dm_repayment").columns:
    new_feature = c + "_numerator"
    hf_parsed[new_feature] = hf_parsed[c].str.replace(r"[^1-7|B]", r"").str.len() 
    hf_parsed[c] = hf_parsed[new_feature] / hf_parsed[c].str.len()

C:\Users\huweipeng\AppData\Local\Temp\ipykernel_808\4203281198.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  hf_parsed[new_feature] = hf_parsed[c].str.replace(r"[^1-7|B]", r"").str.len()
C:\Users\huweipeng\AppData\Local\Temp\ipykernel_808\4203281198.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  hf_parsed[new_feature] = hf_parsed[c].str.replace(r"[^1-7|B]", r"").str.len()
C:\Users\huweipeng\AppData\Local\Temp\ipykernel_808\4203281198.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  hf_parsed[new_feature] = hf_parsed[c].str.replace(r"[^1-7|B]", r"").str.len()
C:\Users\huweipeng\AppData\Local\Temp\ipykernel_808\4203281198.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  hf_parsed[new_feature] = hf_parsed[c].str.replace(r"[^1-7|B]", r"").str.len()


In [120]:
hf_parsed.columns

Index(['id', 'customer_unique_key', 'serial_no', 'create_time', 'edit_time',
       'dm_id_003', 'dm_id_005', 'dm_id_009', 'dm_summary_001',
       'dm_summary_002', 'dm_summary_003', 'dm_summary_004', 'dm_summary_005',
       'dm_summary_006', 'dm_summary_007', 'dm_summary_011', 'dm_summary_012',
       'dm_summary_013', 'dm_summary_014', 'dm_summary_015', 'dm_summary_018',
       'dm_summary_021', 'dm_summary_022', 'dm_summary_023', 'dm_summary_024',
       'dm_summary_025', 'dm_creditdetails_001', 'dm_creditdetails_003',
       'dm_creditdetails_004', 'dm_creditdetails_005', 'dm_creditdetails_006',
       'dm_creditdetails_007', 'dm_creditdetails_009', 'dm_creditdetails_011',
       'dm_creditdetails_012', 'dm_creditdetails_013', 'dm_creditdetails_029',
       'dm_creditdetails_030', 'dm_creditdetails_031', 'dm_creditdetails_032',
       'dm_creditdetails_034', 'dm_publicdetails_001', 'dm_publicdetails_002',
       'dm_publicdetails_003', 'dm_publicdetails_008', 'dm_publicdetails_00

In [121]:
discard = ['dm_summary_022', 'dm_publicdetails_001', 'dm_publicdetails_002', 'dm_publicdetails_003', 'dm_creditdetails_024', 'id', 'customer_unique_key', 'serial_no', 
           'create_time', 'edit_time', 'serial_no.1']
hf_parsed = hf_parsed.drop(columns=discard)

In [122]:
hf_parsed.rename(columns={'label':'overdue'}, inplace=True)

In [ ]:
hf_parsed['fold'] = -1

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for f, (t, v) in enumerate(splitter.split(hf_parsed, hf_parsed['overdue'])):
    hf_parsed.loc[v, 'fold'] = f

In [129]:
hf_parsed.to_csv("./data/hf_parsed_processed.csv", index=False)